In [0]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/spam.csv",delimiter=",",encoding="latin-1")
df.head(3)

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM,Activation,Dense,Dropout,Input,Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline


In [0]:
sns.countplot(df.category)
plt.xlabel('Label')
plt.title("Spam-HAm")

In [0]:
Y = df.category
X = df.message
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [0]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=.2)

In [0]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x_train)
sequences = tok.texts_to_sequences(x_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [0]:
def RNN():
  inputs = Input(name='inputs',shape=[max_len])
  layer = Embedding(max_words,50,input_length=max_len)(inputs)
  layer = LSTM(64)(layer)
  layer = Dense(256,name='FC1')(layer)
  layer = Activation('relu')(layer)
  layer = Dropout(.5)(layer)
  layer = Dense(1,name='out_layer')(layer)
  layer = Activation('sigmoid')(layer)
  model = Model(inputs=inputs,outputs=layer)
  return model

In [0]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc'])


In [0]:
model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,validation_split=.2,
          callbacks=[EarlyStopping(monitor='val_loss',min_delta=.0001)])

In [0]:
test_sequences = tok.texts_to_sequences(x_test)

test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [0]:
accr = model.evaluate(test_sequences_matrix,y_test)

In [0]:
print("Test set loss :{:0.3f}\n accuracy : {:0.3f}".format(accr[0],accr[1]))

In [0]:
tes = ['oh okay ! you got 1 dollar']
t = tok.texts_to_sequences(tes)
t = sequence.pad_sequences(t,maxlen=max_len)

In [0]:
preds = model.predict(t)
print(preds)